In [1]:
import sqlalchemy as db
import pandas as pd
from datetime import datetime
from enum import Enum

class Env(Enum):
    ROOT_FOLDER = '../../data'
    SCHEMA_PATH = '../../oltp/schema.sql'

In [2]:
engine_str = (
      "mysql+pymysql://{user}:{password}@{server}:{port}/{database}".format(
       user      =  "root",
       password  =  "123",
       server    =  "localhost",
       port      =  "3306",
       database  =  "household"))
engine = db.create_engine(engine_str)
conn = engine.connect()

In [9]:
def etl_hh_demographic():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/hh_demographic.csv')
    df['MARITAL_STATUS_CODE'] = df['MARITAL_STATUS_CODE'].map({
                                                        'A': 'Married',
                                                        'B': 'Single',
                                                        'U': 'Unknown'
                                                    })
    return df
    
    # df.to_sql(name="hh_demographic", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)
    

def etl_campaign_desc():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/campaign_desc.csv')
    df = df.rename(columns={'CAMPAIGN': 'CAMPAIGN_ID'})
    return df
    # df.to_sql(name="campaign_desc", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)


def etl_campaign():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/campaign_table.csv')
    df = df.rename(columns={'CAMPAIGN': 'CAMPAIGN_ID'})
    hh_df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/hh_demographic.csv')
    mask = df['household_key'].isin(hh_df['household_key'])
    df = df[mask]
    return df
    # df.to_sql(name="campaign", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)
    

def etl_product():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/product.csv')
    df = df.rename(columns={'MANUFACTURER': 'MANUFACTURER_ID'})
    return df

    # df.to_sql(name="product", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)
    

def etl_causal():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/causal_data.csv')
    df['display'] = df['display'].map({
                                    '0': 'Not on Display',
                                    '1': 'Store Front',
                                    '2': 'Store Rear',
                                    '3': 'Front End Cap',
                                    '4': 'Mid-Aisle End Cap',
                                    '5': 'Rear End Cap',
                                    '6': 'Side-Aisle End Cap',
                                    '7': 'In-Aisle',
                                    '9': 'Sencondary Location Display',
                                    'A': 'In-Shelf'
                                })
    df['mailer'] = df['mailer'].map({
                                    '0': 'Not on ad',
                                    'A': 'Interior page feature',
                                    'C': 'Interior page line item',
                                    'D': 'Front page feature',
                                    'F': 'Back page feature',
                                    'H': 'Wrap front feature',
                                    'J': 'Wrap interior coupon',
                                    'L': 'Wrap back feature',
                                    'P': 'Interior page coupon',
                                    'X': 'Free on interior page',
                                    'Z': 'Free on front page, back page or wrap'
                                })
    return df
    
    # df.to_sql(name="causal", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)
    

def etl_coupon():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/coupon.csv')
    df = df.rename(columns={'CAMPAIGN': 'CAMPAIGN_ID'})
    return df

    # df.to_sql(name="coupon", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)
    
    
def etl_coupon_redempt():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/coupon_redempt.csv')
    df = df.rename(columns={'CAMPAIGN': 'CAMPAIGN_ID'})
    hh_df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/hh_demographic.csv')
    mask = df['household_key'].isin(hh_df['household_key'])
    df = df[mask]
    return df

    # df.to_sql(name="coupon_redempt", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)
    
def etl_transaction():
    df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/transaction_data.csv', dtype={'TRANS_TIME': str})
    hh_df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/hh_demographic.csv')
    mask = df['household_key'].isin(hh_df['household_key'])
    df = df[mask]
    df['TRANS_TIME'] = df['TRANS_TIME'].apply(lambda x: list(str(x)))   \
                                        .apply(lambda x: datetime.strptime(''.join(x[:2] + [':'] + x[2:]), '%H:%M').time())
    return df
    
    # df.to_sql(name="transaction", con=engine,
    #             schema="household",
    #             if_exists = "append", chunksize = 1000,
    #             index=False)

In [34]:
df = pd.read_csv(f'{Env.ROOT_FOLDER.value}/transaction_data.csv')
df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [35]:
new_df = etl_transaction()
new_df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
11,1364,26984896261,1,842930,1,2.19,31742,0.00,15:20:00,1,0.0,0.0
12,1364,26984896261,1,897044,1,2.99,31742,-0.40,15:20:00,1,0.0,0.0
13,1364,26984896261,1,920955,1,3.09,31742,0.00,15:20:00,1,0.0,0.0
14,1364,26984896261,1,937406,1,2.50,31742,-0.99,15:20:00,1,0.0,0.0
15,1364,26984896261,1,981760,1,0.60,31742,-0.79,15:20:00,1,0.0,0.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119384 entries, 0 to 119383
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   COUPON_UPC  119384 non-null  int64
 1   PRODUCT_ID  119384 non-null  int64
 2   CAMPAIGN    119384 non-null  int64
dtypes: int64(3)
memory usage: 2.7 MB


In [33]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119384 entries, 0 to 119383
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype
---  ------       --------------   -----
 0   COUPON_UPC   119384 non-null  int64
 1   PRODUCT_ID   119384 non-null  int64
 2   CAMPAIGN_ID  119384 non-null  int64
dtypes: int64(3)
memory usage: 2.7 MB
